# MIKH Model: Small Amplitude Oscillatory Shear (SAOS)

## Learning Objectives

1. Generate **synthetic SAOS data** from calibrated MIKH parameters
2. Understand the **linear viscoelastic moduli** $G'(\omega)$ and $G''(\omega)$
3. Extract moduli from the MIKH Maxwell backbone
4. Fit and validate the model on oscillatory data
5. Compare with analytical Maxwell predictions

## Prerequisites

- NB01: MIKH Flow Curve (provides calibrated parameters)

## Runtime

- Fast demo: ~3-4 minutes
- Full run: ~10-12 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.ikh import MIKH

# Add examples/utils to path for tutorial utilities
sys.path.insert(0, os.path.join("..", "utils"))
from ikh_tutorial_utils import (
    load_ikh_parameters,
    set_model_parameters,
    generate_synthetic_saos,
    save_ikh_results,
    print_convergence_summary,
    print_parameter_comparison,
    compute_fit_quality,
    get_mikh_param_names,
)

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: SAOS in MIKH

Small Amplitude Oscillatory Shear (SAOS) probes the **linear viscoelastic** response of materials.

### Applied Deformation

$$
\gamma(t) = \gamma_0 \sin(\omega t)
$$

where $\gamma_0 \ll 1$ (typically 0.001-0.01) ensures linear response.

### Stress Response

$$
\sigma(t) = \gamma_0 [G'(\omega) \sin(\omega t) + G''(\omega) \cos(\omega t)]
$$

### Maxwell Model Moduli

For a simple Maxwell element:
$$
G'(\omega) = G \frac{(\omega\tau)^2}{1 + (\omega\tau)^2}
$$
$$
G''(\omega) = G \frac{\omega\tau}{1 + (\omega\tau)^2}
$$

where $\tau = \eta/G$ is the Maxwell relaxation time.

### MIKH Modification

In the MIKH model, the linear viscoelastic response is dominated by the Maxwell backbone, but thixotropy can modify the effective modulus if the structure is not at equilibrium.

## 3. Load Calibrated Parameters

In [ ]:
# Load calibrated parameters from NB01
try:
    calibrated_params = load_ikh_parameters("mikh", "flow_curve")
    print("Loaded calibrated parameters from NB01:")
    for name, val in calibrated_params.items():
        print(f"  {name:15s} = {val:.4g}")
except FileNotFoundError:
    # Provide sensible fallback parameters for SAOS
    print("NB01 results not found. Using fallback parameters suitable for SAOS.")
    calibrated_params = {
        "G": 100.0,           # Shear modulus [Pa]
        "eta": 1e4,           # Viscosity [Pa s] -> tau_M ~ 100 s
        "C": 50.0,            # Kinematic hardening [Pa]
        "gamma_dyn": 1.0,     # Dynamic recovery
        "m": 1.0,             # AF exponent
        "sigma_y0": 5.0,      # Base yield stress [Pa]
        "delta_sigma_y": 20.0,# Structural yield stress [Pa]
        "tau_thix": 10.0,     # Restructuring time [s]
    }

In [ ]:
# Create model and set parameters
model = MIKH()
param_names = get_mikh_param_names()

# Always set parameters (from NB01 or fallback)
set_model_parameters(model, calibrated_params)

# Get Maxwell parameters
G = model.parameters.get_value("G")
eta = model.parameters.get_value("eta")
tau_M = eta / G

print(f"\nMaxwell parameters:")
print(f"  G = {G:.4g} Pa")
print(f"  eta = {eta:.4g} Pa.s")
print(f"  tau_M = {tau_M:.4g} s")
print(f"  omega_crossover = 1/tau_M = {1/tau_M:.4g} rad/s")

## 4. Generate Synthetic Data

In [ ]:
# Generate synthetic SAOS data
omega_range = (0.01, 100.0)
n_points = 30
noise_level = 0.02  # 2% noise

omega, G_prime, G_double_prime = generate_synthetic_saos(
    model,
    omega_range=omega_range,
    n_points=n_points,
    noise_level=noise_level,
    seed=42,
)

print(f"Generated synthetic SAOS data:")
print(f"  Frequency range: [{omega.min():.3f}, {omega.max():.1f}] rad/s")
print(f"  G' range: [{G_prime.min():.2f}, {G_prime.max():.2f}] Pa")
print(f"  G'' range: [{G_double_prime.min():.2f}, {G_double_prime.max():.2f}] Pa")

In [ ]:
# Plot synthetic data with Maxwell reference
omega_fine = np.logspace(np.log10(omega_range[0]), np.log10(omega_range[1]), 100)
wt = omega_fine * tau_M
G_prime_maxwell = G * wt**2 / (1 + wt**2)
G_double_prime_maxwell = G * wt / (1 + wt**2)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(omega, G_prime, "ko", markersize=6, label="$G'$ (data)")
ax.loglog(omega, G_double_prime, "s", color="gray", markersize=6, label="$G''$ (data)")
ax.loglog(omega_fine, G_prime_maxwell, "-", lw=2, color="C0", label="$G'$ (Maxwell)")
ax.loglog(omega_fine, G_double_prime_maxwell, "--", lw=2, color="C1", label="$G''$ (Maxwell)")

# Mark crossover
ax.axvline(1/tau_M, color="red", linestyle=":", alpha=0.5, label=f"$\\omega_c$ = {1/tau_M:.2g} rad/s")

ax.set_xlabel("Angular frequency $\\omega$ [rad/s]", fontsize=12)
ax.set_ylabel("Modulus [Pa]", fontsize=12)
ax.set_title("Synthetic SAOS Data vs Maxwell Model", fontsize=13)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. NLSQ Fitting

We fit the MIKH model to the SAOS data to recover the Maxwell parameters.

In [ ]:
# Prepare complex modulus data (G* = G' + iG'')
G_star = G_prime + 1j * G_double_prime

# Create fresh model for fitting
model_fit = MIKH()

# Fit to oscillation data
# Note: MIKH oscillation fitting uses return mapping with sinusoidal strain
t0 = time.time()
model_fit.fit(omega, np.abs(G_star), test_mode="oscillation", method='scipy')
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in ["G", "eta"]:
    val = model_fit.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

G_fit = model_fit.parameters.get_value("G")
eta_fit = model_fit.parameters.get_value("eta")
tau_fit = eta_fit / G_fit
print(f"\nFitted Maxwell time: tau_M = {tau_fit:.4g} s")
print(f"True Maxwell time:   tau_M = {tau_M:.4g} s")

In [ ]:
# Plot fitted moduli
wt_fit = omega_fine * tau_fit
G_prime_fit = G_fit * wt_fit**2 / (1 + wt_fit**2)
G_double_prime_fit = G_fit * wt_fit / (1 + wt_fit**2)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(omega, G_prime, "ko", markersize=6, label="$G'$ (data)")
ax.loglog(omega, G_double_prime, "s", color="gray", markersize=6, label="$G''$ (data)")
ax.loglog(omega_fine, G_prime_fit, "-", lw=2.5, color="C0", label="$G'$ (MIKH fit)")
ax.loglog(omega_fine, G_double_prime_fit, "--", lw=2.5, color="C1", label="$G''$ (MIKH fit)")

ax.set_xlabel("Angular frequency $\\omega$ [rad/s]", fontsize=12)
ax.set_ylabel("Modulus [Pa]", fontsize=12)
ax.set_title("MIKH SAOS Fit", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 6. Cole-Cole Analysis

In [ ]:
# Cole-Cole plot (G'' vs G')
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(G_prime, G_double_prime, "ko", markersize=6, label="Data")
ax.plot(G_prime_fit, G_double_prime_fit, "-", lw=2, color="C0", label="MIKH fit")

# Maxwell semicircle
theta = np.linspace(0, np.pi, 100)
ax.plot(G_fit/2 * (1 + np.cos(theta)), G_fit/2 * np.sin(theta), "--", 
        color="red", alpha=0.5, label="Maxwell semicircle")

ax.set_xlabel("$G'$ [Pa]", fontsize=12)
ax.set_ylabel("$G''$ [Pa]", fontsize=12)
ax.set_title("Cole-Cole Plot", fontsize=13)
ax.set_aspect("equal")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 7. Bayesian Inference

In [ ]:
# Bayesian inference for Maxwell parameters
initial_values = {name: model_fit.parameters.get_value(name) for name in param_names}

NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
t0 = time.time()
result = model_fit.fit_bayesian(
    omega,
    np.abs(G_star),
    test_mode="oscillation",
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence diagnostics (focus on Maxwell params)
print_convergence_summary(result, ["G", "eta"])

In [ ]:
# Trace plots
idata = result.to_inference_data()
axes = az.plot_trace(idata, var_names=["G", "eta"], figsize=(12, 4))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots (Maxwell Parameters)", fontsize=14, y=1.00)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Parameter comparison
posterior = result.posterior_samples
print_parameter_comparison(model_fit, posterior, ["G", "eta"])

## 8. Physical Interpretation

### Linear Viscoelastic Regime

SAOS probes the **linear viscoelastic** (LVE) properties of the material:
- At small amplitudes, the MIKH model behaves like a Maxwell element
- Yield stress and plasticity don't contribute (below yield)
- Pure elastic ($G'$) and viscous ($G''$) contributions separated

### Crossover Frequency

The crossover frequency $\omega_c = 1/\tau_M = G/\eta$ marks:
- $\omega < \omega_c$: Viscous-dominated ($G'' > G'$)
- $\omega > \omega_c$: Elastic-dominated ($G' > G''$)

### Thixotropy Effects

At low frequencies, thixotropic restructuring during the oscillation cycle can modify the apparent moduli if the oscillation period is comparable to $\tau_{thix}$.

## 9. Save Results

In [ ]:
# Save results
save_ikh_results(model_fit, result, "mikh", "saos", param_names)

## Key Takeaways

1. **SAOS** probes the linear viscoelastic response of MIKH materials

2. The **Maxwell backbone** ($G$, $\eta$) determines the frequency-dependent moduli:
   - $G'(\omega) = G(\omega\tau)^2/(1 + (\omega\tau)^2)$
   - $G''(\omega) = G\omega\tau/(1 + (\omega\tau)^2)$

3. The **crossover frequency** $\omega_c = G/\eta$ separates viscous and elastic regimes

4. **Cole-Cole plots** reveal the Maxwell semicircle for single-mode relaxation

5. SAOS provides clean characterization of $G$ and $\eta$ without yield/plasticity complications

### Next Steps

- **NB06**: LAOS (nonlinear oscillatory response with real PNAS data)